<style>
body {
  background-image: url('background_02.png');
  background-size: cover;
  background-attachment: fixed;
  background-repeat: no-repeat;
  background-position: center;
  margin: 0;
  padding: 0;
}

.main-container {
  background-color: rgba(255, 255, 255, 0.975);
  padding: 30px;
  border-radius: 10px;
  max-width: 900px;
  margin: auto;
  box-shadow: 0px 0px 10px rgba(0,0,0,0.2);
}

/* Optional: Output box styling */
.output-box {
  border: 2px solid #18bc9c;
  background: #eafaf1;
  padding: 12px;
  margin: 1em 0;
  border-radius: 6px;
  font-size: 1.08em;
  font-family: "Fira Mono", "Menlo", "Monaco", "Consolas", monospace;
  white-space: pre-wrap;
}
</style>
<img src="../../Shot_Poses_IACS/OmniacPoses_30.png" width="120" style="float:right; margin: 8px 0 8px 20px;">

# Project Overview

## Objective

This analysis helps **Plurality Labs** and the **Arbitrum DAO** compare strategic framework results on two platforms—**Ethelo** and **Snapshot**. The aim is to determine which channel yields the best quality input. We focus on:

- Comparing contributions by delegate and general contributor wallets
- Revealing similarities and differences in priorities
- Enabling data-driven refinement of the strategic framework
- Sharing all results on a public hub

## Activities & Timeline

- **Data procurement**: Collect aggregate Snapshot data
- **Data processing**: Format and clean for analysis
- **Feature engineering**: Calculate new summary metrics
- **Ranking scheme**: Develop both equal-weighted and token-weighted rankings
- **Analysis**: Compare how priorities differ depending on weighting
- **Sharing**: Publish data and analysis

<img src="../../Shot_Poses_IACS/OmniacPoses_31.png" width="120">

# Background

During #GovMonth, contributors ranked statements related to “Growth and Innovation” and ways Arbitrum could “Reduce Friction”. Respondents ordered statements by preference. The main questions:

**Growth + Innovation Statements**

1. Develop accountability practices within ArbitrumDAO.
2. Identify the key capabilities for improved DAO performance.
3. Form alliances with legacy institutions.
4. Fund projects for cross-chain compatibility.
5. Improve gas fee optimization.
6. Define growth strategies.
7. Incentivize users and builders.
8. Scale the platform.
9. Offer educational opportunities.
10. Evolve governance capabilities.

**Reducing Friction Statements**

1. Build a robust developer community.
2. Make Arbitrum more accessible for developers.
3. Create an inclusive environment.
4. Encourage meaningful DAO participation.
5. Raise awareness of opportunities.
6. Prioritize gas fee optimization.
7. Ensure regulatory compliance.
8. Build anti-Sybil protections.
9. Uphold transparency.
10. Improve token distribution equity.

Each respondent's data includes their wallet, rankings, and Arbitrum holdings (for weighting). Below is a snippet of the GovMonth proposal. For this task, voters were prompted with two sets of statements related to “Growth and Innovation” and ways Arbitrum could “Reduce Friction”. The survey respondents were to rank order each statement according to their personal preference. The statements for each are as follows:

<img src="../../images/govmonth.png" width="600" style="display:block; margin-left:auto; margin-right:auto;">

# Libraries and Setup

Below, we load the necessary Python packages for web requests, data wrangling, and visualization.

In [ ]:
# Load essential libraries
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from datetime import datetime

<div style="text-align: center;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_33.png" width="150"/>
</div>

## Connect to the Snapshot GraphQL API

We use `requests` to create a client for querying the Snapshot API. We'll define the queries for:
- DAO spaces
- Proposals
- Votes

In [ ]:
# Define the Snapshot.org GraphQL endpoint
SNAPSHOT_URL = "https://hub.snapshot.org/graphql"

# Prepare GraphQL queries
SPACE_DATA_QUERY = """
query space_data($skip:Int!){ 
    spaces(orderBy: "id", orderDirection: asc,first:1000,skip:$skip){ 
        id name private about avatar website twitter github coingecko email 
        network symbol domain proposalsCount activeProposals followersCount 
        votesCount verified flagged rank 
    } 
}
"""

PROP_DATA_QUERY = """
query prop_data($slugid: String!, $timestamp: Int!){ 
    proposals(orderBy: "created", orderDirection: asc,first:1000, 
        where:{space:$slugid,created_gt:$timestamp}) { 
        id space{id} ipfs author created network type title body start end 
        state votes choices scores_state scores 
    } 
}
"""

VOTE_DATA_QUERY = """
query vote_data($propid: String!, $timestamp: Int!){ 
    votes(orderBy: "created", orderDirection: asc,first:1000, 
        where:{proposal:$propid,created_gt:$timestamp}) { 
        id proposal{id} ipfs voter created choice vp 
    } 
}
"""

def run_query(query, variables):
    """Function to execute a GraphQL query."""
    response = requests.post(SNAPSHOT_URL, json={'query': query, 'variables': variables})
    response.raise_for_status()
    return response.json()

<div style="text-align: right;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_34.png" width="150"/>
</div>

## Download and Prepare Proposals

We'll now write functions to download:
- All proposals in a space (filtered to two proposals of interest)
- All votes for a proposal

Each function contains step-by-step comments explaining the loop logic and purpose. We then filter the proposals to the two relevant ones for the analysis.

In [ ]:
def get_proposals(slug):
    """Function to fetch all proposals for a given DAO space."""
    c_timestamp = 0
    prop_data = []
    while True:
        variables = {'slugid': slug, 'timestamp': c_timestamp}
        result = run_query(PROP_DATA_QUERY, variables)
        proposals = result.get('data', {}).get('proposals', [])
        if not proposals:
            break
        prop_data.extend(proposals)
        c_timestamp = int(proposals[-1]['created'])
        # print(f"Fetched {len(prop_data)} Entries")
    
    df = pd.json_normalize(prop_data)
    df.rename(columns={'space.id': 'space_id'}, inplace=True)
    return df.drop(columns=['space'])

# Get proposals for Arbitrum DAO
prop_df = get_proposals("arbitrumfoundation.eth")

# Focus on two proposals of interest
proposal_ids = [
    "0x14e71f784e880170972572c2696ef53ef437700c637a151b5176a5827fe5b8bc",
    "0x5824d0b51cc435a49f6455ee2715216d6b958637218ed79e3e93c41af6bdef33"
]
prop_df_sub = prop_df[prop_df['id'].isin(proposal_ids)].copy()


## Download and Prepare Vote Data

This function retrieves all votes for each proposal. Pagination is used to ensure we get all data, and results are stored in a list for each proposal.

In [ ]:
def get_votes(prop_id):
    """Function to fetch all votes for a given proposal ID."""
    c_timestamp = 0
    vote_data = []
    while True:
        variables = {'propid': prop_id, 'timestamp': c_timestamp}
        result = run_query(VOTE_DATA_QUERY, variables)
        votes = result.get('data', {}).get('votes', [])
        if not votes:
            break
        vote_data.extend(votes)
        c_timestamp = int(votes[-1]['created'])
        # print(f"Fetched {len(vote_data)} Entries for proposal {prop_id}") # uncomment to track progress
    df = pd.json_normalize(vote_data)
    df.rename(columns={'proposal.id': 'prop_id'}, inplace=True)
    return df.drop(columns=['proposal'])

# Download votes for both proposals and combine into one dataframe
vote_l = [get_votes(pid) for pid in prop_df_sub['id']]
vote_df = pd.concat(vote_l, ignore_index=True)

<img src="../../Shot_Poses_IACS/OmniacPoses_35.png" width="120" style="float:right; margin: 8px 0 8px 20px;">

# Analyze and Rank Statements

The next step is to calculate various summary statistics for each statement in each proposal, both by unweighted and token-weighted votes. Comments clarify every operation.

In [ ]:
vote_df14 = vote_df[vote_df['prop_id'] == "0x14e71f784e880170972572c2696ef53ef437700c637a151b5176a5827fe5b8bc"].copy()
vote_df58 = vote_df[vote_df['prop_id'] == "0x5824d0b51cc435a49f6455ee2715216d6b958637218ed79e3e93c41af6bdef33"].copy()

def compute_ranking(vote_df):
    """Helper to unpack choices and compute stats."""
    n_choices = 10
    # The 'choice' column is a list of lists, we need to convert it to a NumPy array of ranks
    rankings = np.array([np.argsort(c) + 1 for c in vote_df['choice']])
    
    # Ensure vp is numeric and handle potential NaNs
    vote_df['vp'] = pd.to_numeric(vote_df['vp'], errors='coerce').fillna(0)
    total_vp = vote_df['vp'].sum()
    vp_weights = vote_df['vp'] / total_vp if total_vp > 0 else np.zeros(len(vote_df))

    results = []
    for i in range(1, n_choices + 1):
        # Find where choice 'i' is in each ranking
        ranks_for_choice_i = np.array([np.where(row == i)[0][0] + 1 for row in np.array(list(vote_df['choice']))])
        
        pct_first = np.mean([c[0] == i for c in vote_df['choice']])
        pct_last = np.mean([c[-1] == i for c in vote_df['choice']])
        pct_first_five = np.mean([i in c[:5] for c in vote_df['choice']])
        
        results.append({
            'Choice': i,
            'SumRank': np.sum(ranks_for_choice_i),
            'SumRankAvg': np.mean(ranks_for_choice_i),
            'PctFirst': pct_first,
            'PctLast': pct_last,
            'PctFirstFive': pct_first_five,
            'VPRankAvg': np.sum(ranks_for_choice_i * vp_weights)
        })
    return pd.DataFrame(results)

rankraw14 = compute_ranking(vote_df14)
rankraw58 = compute_ranking(vote_df58)

print("Growth & Innovation Rankings")
display(rankraw14.sort_values(by='SumRank'))
print("\nReducing Friction Rankings")
display(rankraw58.sort_values(by='SumRank'))

<img src="../../Shot_Poses_IACS/OmniacPoses_38.png" width="120" style="float:left; margin: 8px 0 8px 20px;">

## Discussion and Insights
**Example Interpretation**
For "Reducing Friction", "Prioritize Gas Fee Optimization" was ranked highest when votes are equally weighted.

When votes are weighted by tokens, "Build a robust community of developers" rises to the top.

This shows that community and large holders align on the top priorities, but token weighting can shift the relative importance of specific statements.

## Visualizing Rankings

In [ ]:
def plot_ranking(rankdf, title):
    """Function to plot the rankings."""
    df = rankdf.copy()
    df['Statement'] = [f"Choice {i}" for i in df['Choice']]
    df_melted = df.melt(id_vars='Statement', value_vars=['SumRankAvg', 'VPRankAvg'], 
                          var_name='Type', value_name='Value')
    
    plt.figure(figsize=(12, 7))
    sns.barplot(data=df_melted, x='Statement', y='Value', hue='Type', palette='viridis')
    plt.title(title, fontsize=16)
    plt.ylabel('Average Rank')
    plt.xlabel('')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

plot_ranking(rankraw14, "Growth & Innovation: Equal vs Token Weighted Rankings")
plot_ranking(rankraw58, "Reducing Friction: Equal vs Token Weighted Rankings")

<div style="text-align: center;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_36.png" width="150"/>
</div>

# OpenAI Analysis

In this section, we integrate advanced language modeling into our DAO voting analysis. While previous sections relied on deterministic statistical aggregation, here we leverage OpenAI’s GPT-4o-mini model to synthesize a group consensus ranking from raw voting data. This approach introduces a qualitative perspective and allows us to compare traditional quantitative methods with the “collective intelligence” that an LLM might infer from observed voter preferences.

<div style="text-align: right;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_37.png" width="150"/>
</div>

## Extracting Choice Labels Programmatically

Before we can prompt OpenAI, we need to translate the numeric rankings from each voter back to their original statement text. The statement text for each ranked-choice poll is stored in the proposal body field, but not in a direct table. The following function automates extraction by parsing the proposal text for enumerated statements. This ensures our analysis is robust and future-proof, as it adapts to any poll with statements following a consistent numbering format.

In [ ]:
def extract_choice_labels(proposal_body):
    """Extracts the choice labels (statements) from a Snapshot proposal's body text."""
    try:
        # Find the block starting with a variation of the statements header
        start_match = re.search(r"These are the statements:", proposal_body, re.IGNORECASE)
        if not start_match:
            raise ValueError("Can't find statement block in proposal body")
        
        statements_block = proposal_body[start_match.start():]
        
        # Find all lines starting with '1.', '2.', etc.
        lines = statements_block.split('\n')
        choice_lines = [line for line in lines if re.match(r"^\d+\.\s+", line.strip())]
        
        # Remove numbering to get the final labels
        labels = [re.sub(r"^\d+\.\s*", "", line).strip() for line in choice_lines]
        return labels
    except Exception as e:
        print(f"Error extracting labels: {e}")
        return []

# Example: for vote_df14 (growth & innovation)
proposal_id_14 = vote_df14['prop_id'].unique()[0]
proposal_body_14 = prop_df_sub.loc[prop_df_sub['id'] == proposal_id_14, 'body'].iloc[0]
choice_labels_14 = extract_choice_labels(proposal_body_14)

# For vote_df58 (reducing friction)
proposal_id_58 = vote_df58['prop_id'].unique()[0]
proposal_body_58 = prop_df_sub.loc[prop_df_sub['id'] == proposal_id_58, 'body'].iloc[0]
choice_labels_58 = extract_choice_labels(proposal_body_58)

print("--- Growth & Innovation Labels ---")
print(choice_labels_14)
print("\n--- Reducing Friction Labels ---")
print(choice_labels_58)

## Building a Consensus Ranking Prompt for OpenAI
To minimize costs and maximize efficiency when using the OpenAI API, we sample a subset of the available voter rankings. This random sampling reduces the number of tokens sent, while still capturing the main trends in voter sentiment. The following function packages this sample, constructs a precise API prompt, and defines exactly how we expect GPT-4o-mini to return results. Importantly, the prompt asks the model not just for a ranking, but also for a brief explanation of its methodology—providing valuable transparency into its “reasoning” process.

In [ ]:
def get_consensus_ranking(vote_df, prop_df_sub, n_sample=30, api_key_path="your_openai_key.txt"):
    """Main function: Get consensus ranking using OpenAI."""
    # a. Find proposal body for this vote_df
    proposal_id = vote_df['prop_id'].unique()[0]
    proposal_body = prop_df_sub.loc[prop_df_sub['id'] == proposal_id, 'body'].iloc[0]
    if not proposal_body:
        raise ValueError("Proposal body not found or ambiguous!")
    choice_labels = extract_choice_labels(proposal_body)

    # b. Prepare numeric rankings (sample for token efficiency)
    numeric_rankings_text = [','.join(map(str, x)) for x in vote_df['choice']]
    np.random.seed(42)
    sample_indices = np.random.choice(len(numeric_rankings_text), size=min(n_sample, len(numeric_rankings_text)), replace=False)
    sampled_rankings = [numeric_rankings_text[i] for i in sample_indices]
    all_numeric_text = "\n".join(sampled_rankings)

    # Build minimal prompt for OpenAI
    prompt = (
        "You are an expert at group consensus analysis. "
        "Below are multiple ranked lists of choices, each a permutation of the numbers 1–10, where each line is a different voter. "
        "Each number corresponds to a unique statement (but you do not need to know the statements).\n\n"
        "Each line is the ranked order for one voter (first is most preferred, last is least):\n"
        f"{all_numeric_text}\n\n"
        "Your tasks:\n"
        "1. Based only on these rankings, synthesize a single consensus ranking (as a permutation of 1–10) that best reflects the collective preference.\n"
        "2. Output ONLY the consensus ranking as your first line, in the following format (with no explanation before):\n"
        "Consensus: x1,x2,x3,x4,x5,x6,x7,x8,x9,x10\n"
        "3. After the ranking, in 2-4 sentences, explain *how* you synthesized this ranking: mention if you averaged positions, looked for patterns, considered polarization, etc. Focus on what you observed in the rankings, not the meaning of the numbers."
    )

    # Send to OpenAI API
    try:
        with open(api_key_path, 'r') as f:
            openai_api_key = f.read().strip()
    except FileNotFoundError:
        print(f"API key file not found at: {api_key_path}")
        return None

    headers = {
        "Authorization": f"Bearer {openai_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4o-mini",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2,
        "max_tokens": 150
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)

    if response.status_code != 200:
        print(f"Error from OpenAI API: {response.status_code} - {response.text}")
        return None

    # Parse response, map consensus to text
    result = response.json()
    content = result['choices'][0]['message']['content']
    lines = content.strip().split('\n')
    num_line_match = re.search(r"^Consensus:\s*([\d,]+)", lines[0])

    if not num_line_match:
        print("No consensus ranking found in the model output.")
        print(content)
        return None
    
    consensus_numeric_str = num_line_match.group(1)
    consensus_ids = [int(x) for x in consensus_numeric_str.split(',')]
    consensus_statements = [choice_labels[i-1] for i in consensus_ids]

    print("Consensus Ranking (Numbers):\n", consensus_numeric_str, "\n")
    print("Consensus Ranking (Statements):")
    for i, (cid, statement) in enumerate(zip(consensus_ids, consensus_statements)):
        print(f"Rank {i+1}: Choice {cid}: {statement}")
    
    print("\n---\nOpenAI Model Explanation:")
    explanation = "\n".join(lines[1:]).strip()
    print(explanation, "\n")
    return consensus_statements

<div style="text-align: left;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_39.png" width="150"/>
</div>

## Calling the GPT-4o-mini Consensus Function

Now, we call our custom function, passing in the appropriate vote and proposal data. The output will include both the ranked statements (mapped from the original choice indices) and the model's brief explanation of its ranking strategy.

By comparing these results to our deterministic, token-weighted rankings, we can highlight the differences between a purely statistical aggregation and the type of pattern recognition and consensus modeling performed by a state-of-the-art language model. This comparison adds another layer of interpretability to our study and offers a compelling way to triangulate DAO preferences.

*Note: API costs can scale with prompt size, so be mindful of your sample size and your OpenAI account limits.*

### Interpretation
The GPT-4o-mini model's ranking is sometimes different from our classic deterministic aggregation. This is partly due to random sampling (to save tokens/cost) but also reflects the LLM's ability to weigh not only strict averages but also patterns in the distribution of ranks—such as consistency, outliers, and relative ordering trends. Its brief "reasoning" section can highlight patterns we might otherwise miss, making it a useful supplement to classic data science methods.

In [ ]:
# -- For proposal 14 (Growth & Innovation) --
print("--- Analyzing Growth & Innovation ---")
consensus_14 = get_consensus_ranking(vote_df14, prop_df_sub, n_sample=500, api_key_path="../../openai_key.txt")

# -- For proposal 58 (Reducing Friction) --
print("\n--- Analyzing Reducing Friction ---")
consensus_58 = get_consensus_ranking(vote_df58, prop_df_sub, n_sample=500, api_key_path="../../openai_key.txt")

<div style="text-align: right;">
  <img src="../../Shot_Poses_IACS/OmniacPoses_41.png" width="150"/>
</div>

# Conclusions

This analysis provides a comprehensive framework for quantifying and understanding alignment between large token holders and the broader Arbitrum DAO community with respect to strategic priorities. By examining both equal-weighted (one wallet, one vote) and token-weighted (voting power) rankings, we are able to reveal not only the overall consensus but also to highlight potential divergences between high-stake holders and general participants.

Through classic statistical aggregation, we demonstrated which statements consistently ranked at the top across different voting schemes, thereby identifying areas of strong community consensus. Conversely, discrepancies between the two schemes can help surface priorities that are particularly important to either major stakeholders or the broader base, informing more inclusive decision-making processes.

In addition to these quantitative approaches, we integrated OpenAI's state-of-the-art language model (GPT-4o-mini) into our workflow. By prompting the AI with real anonymized voting records, we obtained an AI-synthesized consensus ranking and a concise explanation of the logic behind the ordering. This adds a qualitative layer of analysis that can capture subtle patterns—such as clustering, polarization, or emergent trends in ranking behavior—that might be overlooked by traditional methods. The AI explanation also enhances transparency and interpretability, offering an alternative lens for both researchers and governance participants.

Our methodology is generalizable and can be readily applied to other DAO governance polls and ranking-based decision scenarios. By blending statistical analysis with AI-based synthesis, we offer a richer, multidimensional perspective on community preferences—empowering DAOs to make more informed, balanced, and democratic decisions.

*In summary, combining traditional ranking analytics with modern AI tools not only validates results through multiple lenses but also brings greater transparency and insight into complex governance ecosystems like Arbitrum DAO.*